In [ ]:
#!python
from libs import *

In [ ]:
request_times = [0,0]

In [ ]:
def historical_data(symbol, from_date, to_date, interval):

    """
    This function will return historical data of the instrument for specific period of days for specific interval
    """
    
    df = pd.DataFrame()   
    int_token = instrument_token(master_df, symbol)  #the function we defined above which will return token no. of instrument
    
    to_date   = pd.Timestamp(to_date)
    from_date = pd.Timestamp(from_date)
    
    limit_dict = {'minute' : 60, '3minute' : 100, '5minute' : 100, '10minute' : 100, '15minute' : 200, '30minute' : 200, '60minute' : 400, 'day' : 2000}
    limit = limit_dict[interval]
    while True:
        if from_date >= (to_date - dt.timedelta(limit)):
            #if from_date is within the 60 days limit
            while time.time() - request_times[-2] < 1:
                pass
            # print(int_token, from_date, to_date, interval)
            while True:
                try:
                    # Fetch historical data
                    temp_df = pd.DataFrame(kite.historical_data(int_token, from_date, to_date, interval))
                    break
                except exceptions.DataException:
                    time.sleep(1)

            request_times.append(time.time())
            df = pd.concat([df, temp_df])
            break
            
        else:                                                            #if from_date has more than 60 days limit
            to_date_new = from_date + dt.timedelta(limit)
            while time.time() - request_times[-2] < 1:
                pass
            # print(int_token, from_date, to_date_new, interval)
            while True:
                try:
                    # Fetch historical data
                    temp_df = pd.DataFrame(kite.historical_data(int_token, from_date, to_date_new, interval))
                    break
                except exceptions.DataException:
                    time.sleep(1)
            request_times.append(time.time())
            df = pd.concat([df, temp_df])
            #to_date = from_date.date() + dt.timedelta(60)
            from_date = to_date_new
            
    return df

In [ ]:
#paths
trading_ = Path('/Users/yash/Desktop/Trading')
code_ = trading_ / "code"
minute_ = trading_ / "historical_data/kite_connect/minute"
minute_5 = trading_ / "historical_data/kite_connect/minute_5"
nifty500 = trading_ / "tradingview_watchlists/broad/ind_nifty500list.csv"
df500 = pd.read_csv(nifty500)
nifty500_list = df500['Symbol']

In [ ]:
connection_file = code_ / "config/connection_info.json"
conInfo = get_connection_info(connection_file)
kite = login_in_zerodha(conInfo['KITE_API_KEY'],
                        conInfo['KITE_API_SECRET'],
                        conInfo['USER'],
                        conInfo['PASSWORD'],
                        conInfo['TOTP_KEY'])
print(kite.profile())

In [ ]:
nse_df = pd.DataFrame(kite.instruments('NSE'))
bse_df = pd.DataFrame(kite.instruments('BSE'))
master_df = pd.concat([nse_df, bse_df])
master_df.drop_duplicates(subset="tradingsymbol",
                     keep='first', inplace=True)
master_df.drop_duplicates(subset="name",
                     keep='first', inplace=True)

In [ ]:
start_ = '2022-01-01'
end_ = '2022-12-31'

In [ ]:
# gauth = GoogleAuth(settings_file=code_ / "config/settings.yaml")           
# drive = GoogleDrive(gauth)

In [ ]:
# #list
# done_set = set()
# minute_folder = '1tc6yb3kkqmgyDh7am7SU17_FupTlBg9o'
# drive_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(minute_folder)}).GetList()

In [ ]:
# for file in drive_list:
#     title = file['title']
#     id = file['id']
#     # print(title)
#     # break
#     if 'yash' in title:
#         newTitle = title.split('/')[-1]
#         print(newTitle)
#         # renameFile(drive, id, newTitle)
#         a=drive.auth.service.files().get(fileId=id).execute()
#         a['title']=newTitle
#         update=drive.auth.service.files().update(fileId=id,body=a).execute()

In [ ]:
# print('Checking in GDrive')
# done_set = set()
# for file in drive_list:
#     # print(file['title'])
#     # break
#     done_set.add(file['title'].split('_')[0])
# print(len(done_set))
# files = glob('/Users/yash/Desktop/Trading/historical_data/kite_connect/minute/*')
# print('Checking in local machine')
# for file in files:
#     # print(file)
#     # break
#     done_set.add(file.split('/')[-1].split('_')[0])
# print(len(done_set))

In [ ]:
interval = "5minute"

In [ ]:
os.chdir(minute_5)

In [ ]:
done_set = set()

In [ ]:
for symbol in nifty500_list:
    try:
        # break
        fname = symbol + '_' + start_.replace('-','') + '_' + end_.replace('-','')
        if symbol in done_set:
            done_set.add(symbol)
            print('Skipping ', symbol)
            continue
        # print(fname)
        df = historical_data(symbol, start_, end_, interval)
        # break
        df.to_csv(fname)
        done_set.add(symbol)

        # files = glob(minute_.name + '*')
        # while len(files) > 500:
        #     print('Waiting')
        #     pass
        print(symbol)
    except:
        print('Error: ', symbol)

NIFTY 500